In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import fbeta_score

In [2]:
datafile_train=r'D:/Pryanka/Edvancer/data/carvan_train.csv'
datafile_test=r'D:/Pryanka/Edvancer/data/carvan_test.csv'
cd_train=pd.read_csv(datafile_train)
cd_test=pd.read_csv(datafile_test)

In [3]:
print(cd_train.shape)
cd_train.isnull().sum().sum(),cd_test.isnull().sum().sum() 

(5822, 86)


(0, 0)

In [4]:
target='V86'
cd_train[target].value_counts()

V86
0    5474
1     348
Name: count, dtype: int64

In [5]:
x_train=cd_train.drop(target,axis=1)
y_train=cd_train[target]

In [6]:
model=LogisticRegression(fit_intercept=True,max_iter=5000)

In [7]:
params={'penalty':['l1','l2'],
        'C':np.linspace(0.01,100,20),
       #  'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
           'max_iter' : [100, 1000,2500, 5000],
        'class_weight':['balanced',None]}

In [8]:
feature_cols = ['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24','V25','V26','V27','V28','V29','V30','V31','V32','V33','V34','V35','V36','V37','V38','V39','V40','V41','V42','V43','V44','V45','V46','V47','V48','V49','V50','V51','V52','V53','V54','V55','V56','V57','V58','V59','V60','V61','V62','V63','V64','V65','V66','V67','V68','V69','V70','V71','V72','V73','V74','V75','V76','V77','V78','V79','V80','V81','V82','V83','V84','V85']
X = cd_train[feature_cols] # Features
y = cd_train.V86 # Target variable

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=4000)

In [10]:
X_train.shape
y_train.shape
model.fit(X_train, y_train)

LogisticRegression(max_iter=5000)

In [11]:
model.score(X_train,y_train)

0.940219880897847

In [12]:
clf = GridSearchCV(estimator = model, param_grid = params, scoring = 'roc_auc', cv = 10, verbose=True, n_jobs=-1)

In [13]:
clf_fit = clf.fit(X_train,y_train)

Fitting 10 folds for each of 320 candidates, totalling 3200 fits


In [14]:
clf_fit.best_estimator_

LogisticRegression(C=0.01, class_weight='balanced')

In [15]:
clf_fit.score(X_train,y_train)

0.7959516613442823

In [16]:
clf_fit.best_score_

0.7564579824068877

In [17]:
train_score=clf_fit.best_estimator_.predict_proba(X_train)[:,1]

In [18]:
real=y_train

In [19]:
cutoffs=np.linspace(0.001,0.999,999)

In [20]:
from sklearn.metrics import fbeta_score

In [21]:
fbetas=[]

In [22]:
for cutoff in cutoffs:
    
    predicted=(train_score>cutoff).astype(int)
    
    fbetas.append(fbeta_score(y_train,predicted,beta=2.0))

In [23]:
my_cutoff=cutoffs[fbetas==max(fbetas)]

In [24]:
predictions=(clf_fit.predict_proba(cd_test)[:,1]>my_cutoff).astype(int)

In [25]:
pd.Series(predictions).value_counts()

0    2969
1    1031
Name: count, dtype: int64

In [26]:
submissions=pd.DataFrame({'V86':predictions})

In [27]:
submissions.to_csv('D:/Pryanka/Edvancer/BFSI_submission.csv',index=False)